## Ingest Drivers.json file

In [0]:
%fs
ls /mnt/prisha12/raw/

### 1. reading csv file, trying to fiugre out how is the format

In [0]:
#nested JSON shcema for name 
from pyspark.sql.types import StructType,StructField,StringType,IntegerType, TimestampType,DateType
name_schema = StructType([StructField('forename',StringType(),True),StructField('surname',StringType(),True)])
driver_schema =StructType([StructField('driverId',IntegerType(),False),
                          StructField('driverRef',StringType(),True),
                          StructField('number',IntegerType(),True),
                          StructField('code',StringType(),True),
                          StructField('name',name_schema,True),
                          StructField('dob',DateType(),True),
                          StructField('nationality',StringType(),True),
                          StructField('url',StringType(),True)])
                           

In [0]:
df = spark.read.schema(driver_schema).json("/mnt/prisha12/raw/drivers.json")
display(df.limit(10))

### renamed columns and add new columns ingesting date, concatenate forname and surname


In [0]:
from pyspark.sql.functions import current_timestamp, col, concat, lit
drivers_df = df.withColumnRenamed('driverId','driver_id') \
                         .withColumnRenamed('driverRef','driver_ref') \
                         .withColumn("ingestion_date", current_timestamp()) \
                         .withColumn('name',concat(col("name.forename"),lit(" "),col("name.surname")))
display(drivers_df.limit(10))

### drop column url

In [0]:
drivers_df=drivers_df.drop('url')

In [0]:
##write this data into data lake processed
drivers_df.write.mode("overwrite").format("parquet").save("/mnt/prisha12/processed/drivers")

In [0]:
%fs
ls /mnt/prisha12/processed/drivers